# Lezione 34 — Document Intelligence

## BLOCCO 4: Artificial Intelligence & NLP

---

## Obiettivi di Apprendimento

Al termine di questa lezione sarai in grado di:

1. **Definire** cosa si intende per Document Intelligence
2. **Classificare** documenti automaticamente per tipologia
3. **Estrarre** informazioni strutturate da documenti non strutturati
4. **Implementare** un sistema di document processing con sklearn
5. **Comprendere** il workflow end-to-end di document automation

---

## Prerequisiti

- Lezione 30-31: Rappresentazione testo (TF-IDF)
- Lezione 32: Sentiment Analysis (classificazione)
- Lezione 33: Named Entity Recognition

---

## Indice

1. Teoria — Cos'è la Document Intelligence
2. Tipologie di Task
3. Schema Mentale
4. Notebook Dimostrativo
5. Esercizi Svolti
6. Conclusione e Bignami

---

# 1. Teoria — Cos'è la Document Intelligence

## 1.1 Definizione

La **Document Intelligence** (o Intelligent Document Processing - IDP) è l'applicazione di AI per automatizzare l'estrazione, la classificazione e l'elaborazione di informazioni da documenti.

```
Input: Documenti non strutturati (PDF, email, scan)
       ↓
Processing: OCR + NLP + ML
       ↓
Output: Dati strutturati (JSON, database)
```

## 1.2 Componenti Chiave

| Componente | Funzione | Tecnologie |
|------------|----------|------------|
| **OCR** | Riconoscimento testo da immagini | Tesseract, Azure Form Recognizer |
| **Document Classification** | Categorizzazione tipo documento | Text classification, CNN |
| **Key-Value Extraction** | Estrarre campi specifici | NER, Template matching |
| **Table Extraction** | Identificare e parsare tabelle | Layout analysis |
| **Validation** | Verificare consistenza dati | Business rules |

## 1.3 Perché è Importante

**Problema Aziendale:**
- 80% delle informazioni aziendali sono in documenti non strutturati
- Elaborazione manuale è lenta, costosa, error-prone

**Soluzione IDP:**
- Automazione 70-90% del processing
- Riduzione errori umani
- Scalabilità

## 1.4 Casi d'Uso Comuni

| Settore | Documento | Estrazione |
|---------|-----------|------------|
| Finance | Fatture | Importo, data, fornitore, voci |
| Insurance | Sinistri | Data, descrizione, danni, testimoni |
| Legal | Contratti | Parti, clausole, date, condizioni |
| HR | CV | Nome, competenze, esperienze |
| Healthcare | Referti | Diagnosi, prescrizioni, valori |

---

## 1.5 Task Principali di Document Intelligence

### Task 1: Document Classification

Classificare il tipo di documento:
- È una fattura, un contratto, una lettera?
- È urgente, normale, spam?

```
Input: "Fattura N. 123 del 01/01/2024..."
Output: {"tipo": "fattura", "confidence": 0.95}
```

### Task 2: Key-Value Extraction (Form Understanding)

Estrarre campi specifici da un form:

```
Input: Fattura scansionata
Output: {
    "numero_fattura": "123",
    "data": "01/01/2024",
    "fornitore": "Acme S.r.l.",
    "totale": "€1.250,00"
}
```

### Task 3: Table Extraction

Identificare e parsare tabelle:

```
Input: PDF con tabelle
Output: [
    {"descrizione": "Servizio A", "quantità": 10, "prezzo": 100},
    {"descrizione": "Servizio B", "quantità": 5, "prezzo": 200}
]
```

### Task 4: Document Summarization

Riassumere contenuto di documenti lunghi:

```
Input: Contratto di 50 pagine
Output: "Contratto di fornitura servizi IT tra X e Y, 
        durata 3 anni, valore €500k/anno..."
```

## 1.6 Approcci Tecnici

| Approccio | Come Funziona | Pro/Contro |
|-----------|---------------|------------|
| **Template-based** | Pattern fissi per ogni tipo doc | Preciso ma rigido |
| **ML Traditional** | TF-IDF + Classificatori | Flessibile, richiede dati |
| **Deep Learning** | CNN per layout, Transformers per testo | Massima accuracy, GPU |
| **Hybrid** | Regole + ML | Bilanciato |

---

# 2. Schema Mentale — Pipeline IDP

## Workflow End-to-End

```
                    DOCUMENT INTELLIGENCE PIPELINE
                    
┌─────────────────────────────────────────────────────────────┐
│                    INGESTION                                │
│  PDF, Scan, Email, Immagini → Acquisizione documenti        │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                    PREPROCESSING                            │
│  ┌───────────┐  ┌───────────┐  ┌───────────┐               │
│  │   OCR     │  │  Cleaning │  │   Layout  │               │
│  │ (se scan) │  │ (noise)   │  │  Analysis │               │
│  └───────────┘  └───────────┘  └───────────┘               │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                   CLASSIFICATION                            │
│                                                             │
│       "Che tipo di documento è?"                            │
│                                                             │
│    Fattura │ Contratto │ Lettera │ Report │ Altro          │
│       ↓                                                     │
│    Route a pipeline specifica                               │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                    EXTRACTION                               │
│                                                             │
│   ┌─────────────────────────────────────────────────────┐  │
│   │ Fattura Pipeline:                                    │  │
│   │   → Estrai: numero, data, fornitore, righe, totale  │  │
│   └─────────────────────────────────────────────────────┘  │
│   ┌─────────────────────────────────────────────────────┐  │
│   │ Contratto Pipeline:                                  │  │
│   │   → Estrai: parti, date, clausole, valore           │  │
│   └─────────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                    VALIDATION                               │
│                                                             │
│   • Campi obbligatori presenti?                             │
│   • Formati corretti? (date, importi)                       │
│   • Valori plausibili? (totale = somma righe)               │
│   • Confidence score accettabile?                           │
│                                                             │
│   → Se fallisce: Human-in-the-loop review                   │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                     OUTPUT                                  │
│                                                             │
│   JSON / Database / ERP / Workflow                          │
│                                                             │
│   {                                                         │
│     "tipo": "fattura",                                      │
│     "numero": "F-2024-001",                                 │
│     "data": "2024-01-15",                                   │
│     "fornitore": "Acme S.r.l.",                             │
│     "totale": 1250.00,                                      │
│     "confidence": 0.94                                      │
│   }                                                         │
└─────────────────────────────────────────────────────────────┘
```

## Decisioni Architetturali

```
           SCELTA DELL'APPROCCIO
           
     Volume documenti?
            │
   ┌────────┴────────┐
   │ <100/mese       │ >1000/mese
   ▼                 ▼
Template       Machine Learning
+ Regex        + Deep Learning
(basso costo)  (alto ROI su scala)
```

---

# 3. Notebook Dimostrativo

## Setup e Dataset Documenti

In [ ]:
# ============================================================
# DEMO 1: Dataset di Documenti Aziendali
# ============================================================

import numpy as np
import pandas as pd
import re
from collections import namedtuple
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Simuliamo un corpus di documenti aziendali
documenti = [
    # FATTURE (10)
    {"tipo": "fattura", "testo": "FATTURA N. 2024/001 Data: 15/01/2024 Cliente: Rossi S.r.l. Totale: €1.250,00 IVA inclusa"},
    {"tipo": "fattura", "testo": "Fattura numero F-123 del 20/02/2024. Spett.le Bianchi SpA. Importo totale €3.500,00"},
    {"tipo": "fattura", "testo": "INVOICE #456 Date: 2024-03-10 To: Verdi & Co. Amount: €750.00 VAT included"},
    {"tipo": "fattura", "testo": "Fattura di vendita n. 789 Destinatario: Mario Neri P.IVA 12345678901 Totale €2.100,00"},
    {"tipo": "fattura", "testo": "Documento fiscale - Fattura 2024/100 Importo imponibile €1.000 IVA 22% €220 Totale €1.220"},
    {"tipo": "fattura", "testo": "FATTURA ELETTRONICA SDI N. IT12345 Cliente: Alfa Beta Srl Totale fattura: €4.800,00"},
    {"tipo": "fattura", "testo": "Invoice from TechCorp Amount due: €2.500 Payment terms: 30 days Due date: 15/04/2024"},
    {"tipo": "fattura", "testo": "Fattura di cortesia n. 55 Servizi consulenza Q1 2024 Compenso €6.000 oltre IVA"},
    {"tipo": "fattura", "testo": "Nota di debito n. 12 riferita a fattura 2024/050 Maggiorazione €150,00"},
    {"tipo": "fattura", "testo": "Fattura proforma n. 001 Anticipo 50% importo €500 Saldo €500 alla consegna"},
    
    # CONTRATTI (10)
    {"tipo": "contratto", "testo": "CONTRATTO DI FORNITURA tra Alfa SpA e Beta Srl durata 36 mesi decorrenza 01/01/2024"},
    {"tipo": "contratto", "testo": "Accordo di collaborazione Le parti convengono quanto segue clausola 1 clausola 2 firma"},
    {"tipo": "contratto", "testo": "CONTRATTO DI LICENZA SOFTWARE Licenziante: TechCorp Licenziatario: Cliente Canone annuo €10.000"},
    {"tipo": "contratto", "testo": "Agreement between parties A and B Terms and conditions Duration: 24 months Termination clause"},
    {"tipo": "contratto", "testo": "Contratto di consulenza Il Consulente si impegna a Le parti concordano Penali ritardo"},
    {"tipo": "contratto", "testo": "CONTRATTO DI LOCAZIONE Locatore Conduttore Canone mensile €1.200 Deposito cauzionale"},
    {"tipo": "contratto", "testo": "Mandato professionale Oggetto dell'incarico Compenso pattuito Clausola riservatezza"},
    {"tipo": "contratto", "testo": "Contratto di appalto Committente: Comune di Milano Appaltatore: Costruzioni Srl Importo lavori"},
    {"tipo": "contratto", "testo": "PATTO DI NON CONCORRENZA Il dipendente si impegna a non svolgere attività concorrenziali"},
    {"tipo": "contratto", "testo": "Contratto di agenzia Preponente Agente Provvigioni 5% sul fatturato Zona esclusiva"},
    
    # LETTERE (10)
    {"tipo": "lettera", "testo": "Gentile Dott. Rossi, in riferimento alla Sua richiesta, Le comunichiamo che Distinti saluti"},
    {"tipo": "lettera", "testo": "Egregio Direttore, con la presente desidero sottoporre alla Sua attenzione Cordiali saluti"},
    {"tipo": "lettera", "testo": "Dear Sir/Madam, I am writing to inform you that Yours sincerely John Smith"},
    {"tipo": "lettera", "testo": "Spett.le Ufficio Reclami, mi vedo costretto a segnalare un disservizio In attesa di riscontro"},
    {"tipo": "lettera", "testo": "Alla cortese attenzione del Responsabile Oggetto: richiesta informazioni La ringrazio"},
    {"tipo": "lettera", "testo": "Gentilissima Dott.ssa Bianchi, Le scrivo per confermare la nostra disponibilità A presto"},
    {"tipo": "lettera", "testo": "Caro Collega, ti informo che la riunione è stata spostata Saluti Mario"},
    {"tipo": "lettera", "testo": "Spett.le Direzione, con riferimento alla circolare del Mi permetto di suggerire"},
    {"tipo": "lettera", "testo": "Egregi Signori, alleghiamo documentazione richiesta Rimaniamo a disposizione"},
    {"tipo": "lettera", "testo": "To whom it may concern, This letter is to certify that Best regards HR Department"},
    
    # REPORT (10)
    {"tipo": "report", "testo": "REPORT TRIMESTRALE Q1 2024 Fatturato: €1.5M EBITDA: €300k Crescita YoY: +15%"},
    {"tipo": "report", "testo": "Analisi di mercato Sintesi esecutiva Trend principali Raccomandazioni Conclusioni"},
    {"tipo": "report", "testo": "RELAZIONE TECNICA Oggetto: impianto elettrico Descrizione intervento Materiali Collaudo"},
    {"tipo": "report", "testo": "Report settimanale KPI vendite Obiettivi raggiunti Criticità riscontrate Azioni correttive"},
    {"tipo": "report", "testo": "Quarterly Business Review Revenue: €2M Pipeline: €5M Win rate: 35% Forecast Q2"},
    {"tipo": "report", "testo": "AUDIT REPORT Internal controls assessment Findings Recommendations Risk rating"},
    {"tipo": "report", "testo": "Relazione di perizia Premessa Analisi Valutazione conclusiva €500.000"},
    {"tipo": "report", "testo": "Report progetto Status: On track Milestone completate Budget utilizzato 60%"},
    {"tipo": "report", "testo": "Sintesi riunione CdA Delibere approvate Prossimi passi Calendario riunioni"},
    {"tipo": "report", "testo": "Annual Report 2023 Highlights Key figures Chairman letter Financial statements"},
]

# Creazione DataFrame
df = pd.DataFrame(documenti)

print("="*60)
print("DATASET DOCUMENTI AZIENDALI")
print("="*60)
print(f"\nTotale documenti: {len(df)}")
print(f"\nDistribuzione per tipo:")
print(df['tipo'].value_counts())
print(f"\nEsempi per tipo:")
for tipo in df['tipo'].unique():
    esempio = df[df['tipo'] == tipo].iloc[0]['testo'][:60]
    print(f"  {tipo.upper()}: '{esempio}...'")

In [ ]:
# ============================================================
# DEMO 2: Document Classification con TF-IDF
# ============================================================
# Classifichiamo automaticamente il tipo di documento

# Preparazione dati
X = df['testo'].values
y = df['tipo'].values

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Pipeline TF-IDF + Classifier
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=500,
    lowercase=True
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Classificatore
classifier = LogisticRegression(max_iter=1000, random_state=42)
classifier.fit(X_train_tfidf, y_train)

# Valutazione
y_pred = classifier.predict(X_test_tfidf)

print("="*60)
print("DEMO 2: DOCUMENT CLASSIFICATION")
print("="*60)
print(f"\nAccuracy: {(y_pred == y_test).mean():.2%}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print(f"Confusion Matrix:")
print("-" * 40)
cm = confusion_matrix(y_test, y_pred)
labels = sorted(df['tipo'].unique())
print(f"{'':12}", end='')
for l in labels:
    print(f"{l[:8]:>10}", end='')
print()
for i, l in enumerate(labels):
    print(f"{l[:10]:12}", end='')
    for j in range(len(labels)):
        print(f"{cm[i,j]:>10}", end='')
    print()

In [ ]:
# ============================================================
# DEMO 3: Parole Chiave per Tipo Documento
# ============================================================
# Quali termini distinguono ogni tipo di documento?

feature_names = vectorizer.get_feature_names_out()

# Per ogni classe, trova i termini più discriminanti
print("="*60)
print("DEMO 3: TERMINI DISTINTIVI PER TIPO")
print("="*60)

# Coefficienti per ogni classe
# Per multiclass, abbiamo coefficienti per ogni classe
classes = classifier.classes_

for i, classe in enumerate(classes):
    # I coefficienti per questa classe
    coeffs = classifier.coef_[i]
    
    # Top 10 termini per questa classe
    top_indices = np.argsort(coeffs)[::-1][:8]
    
    print(f"\n📄 {classe.upper()}:")
    print("  Termini distintivi:")
    for idx in top_indices:
        print(f"    • {feature_names[idx]:20} (coef: {coeffs[idx]:+.3f})")

# Insight
print(f"\n{'='*60}")
print("INSIGHT")
print("="*60)
print("""
I coefficienti mostrano quali termini "attivano" ogni classe:
• FATTURA: termini come "fattura", "importo", "iva", "totale"
• CONTRATTO: termini come "contratto", "parti", "clausola"
• LETTERA: termini come "gentile", "saluti", "riferimento"
• REPORT: termini come "report", "analisi", "trend", "kpi"

Questo è interpretabile: possiamo spiegare perché un documento
viene classificato in un certo modo.
""")

In [ ]:
# ============================================================
# DEMO 4: Key-Value Extraction per Fatture
# ============================================================
# Estraiamo campi specifici dalle fatture

from typing import Dict, List, Optional
from dataclasses import dataclass

@dataclass
class InvoiceData:
    """Struttura dati per fattura estratta."""
    numero: Optional[str] = None
    data: Optional[str] = None
    cliente: Optional[str] = None
    totale: Optional[str] = None
    iva: Optional[str] = None
    confidence: float = 0.0

class InvoiceExtractor:
    """
    Estrattore di dati da testo fattura.
    Usa regex per campi strutturati.
    """
    
    def __init__(self):
        # Pattern per ogni campo
        self.patterns = {
            'numero': [
                r'(?:fattura|invoice|n\.?|numero|#)\s*[:.]?\s*([A-Z0-9\-/]+)',
                r'(?:n\.?|numero)\s*(?:fattura)?\s*[:.]?\s*([A-Z0-9\-/]+)',
            ],
            'data': [
                r'(?:data|date|del)\s*[:.]?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})',
                r'(\d{4}-\d{2}-\d{2})',  # ISO format
            ],
            'totale': [
                r'(?:totale|total|amount|importo)\s*[:.]?\s*(€?\s*[\d.,]+(?:\s*(?:€|euro))?)',
                r'(€\s*[\d.,]+)',
            ],
            'cliente': [
                r'(?:cliente|customer|spett\.?le|to)\s*[:.]?\s*([A-Za-zÀ-ÿ\s&\.]+(?:s\.?r\.?l\.?|s\.?p\.?a\.?)?)',
            ],
            'iva': [
                r'(?:iva|vat)\s*[:.]?\s*([\d.,]+%?|inclusa|esclusa)',
            ],
        }
    
    def extract(self, testo: str) -> InvoiceData:
        """Estrae dati strutturati da testo fattura."""
        result = InvoiceData()
        campi_trovati = 0
        campi_totali = len(self.patterns)
        
        for campo, patterns in self.patterns.items():
            for pattern in patterns:
                match = re.search(pattern, testo, re.IGNORECASE)
                if match:
                    valore = match.group(1).strip()
                    setattr(result, campo, valore)
                    campi_trovati += 1
                    break  # Usa primo match
        
        # Confidence basata su quanti campi trovati
        result.confidence = campi_trovati / campi_totali
        
        return result
    
    def validate(self, data: InvoiceData) -> Dict:
        """Valida i dati estratti."""
        issues = []
        
        if not data.numero:
            issues.append("Numero fattura mancante")
        if not data.data:
            issues.append("Data fattura mancante")
        if not data.totale:
            issues.append("Totale mancante")
        if data.confidence < 0.6:
            issues.append(f"Bassa confidence: {data.confidence:.0%}")
        
        return {
            'valid': len(issues) == 0,
            'issues': issues,
            'requires_review': data.confidence < 0.8
        }

# Test
extractor = InvoiceExtractor()

fatture_test = [
    "FATTURA N. 2024/001 Data: 15/01/2024 Cliente: Rossi S.r.l. Totale: €1.250,00 IVA inclusa",
    "Invoice #456 Date: 2024-03-10 To: Verdi & Co. Amount: €750.00 VAT included",
    "Fattura numero F-789 Spett.le Bianchi SpA Importo €3.500",
]

print("="*60)
print("DEMO 4: EXTRACTION DA FATTURE")
print("="*60)

for i, fattura in enumerate(fatture_test, 1):
    print(f"\n📄 Fattura {i}:")
    print(f"   Testo: '{fattura[:60]}...'")
    
    # Estrazione
    data = extractor.extract(fattura)
    
    print(f"\n   Campi estratti:")
    print(f"      Numero:  {data.numero or 'N/A'}")
    print(f"      Data:    {data.data or 'N/A'}")
    print(f"      Cliente: {data.cliente or 'N/A'}")
    print(f"      Totale:  {data.totale or 'N/A'}")
    print(f"      IVA:     {data.iva or 'N/A'}")
    print(f"      Confidence: {data.confidence:.0%}")
    
    # Validazione
    validation = extractor.validate(data)
    status = "✅ VALIDO" if validation['valid'] else "⚠️ PROBLEMI"
    print(f"\n   Validazione: {status}")
    for issue in validation['issues']:
        print(f"      - {issue}")

In [ ]:
# ============================================================
# DEMO 5: Pipeline Integrata Document Intelligence
# ============================================================
# Combiniamo Classification + Extraction in un'unica pipeline

class DocumentIntelligencePipeline:
    """
    Pipeline completa per Document Intelligence:
    1. Classifica il documento
    2. Applica extractor specifico per tipo
    3. Valida e restituisce risultato strutturato
    """
    
    def __init__(self, vectorizer, classifier):
        self.vectorizer = vectorizer
        self.classifier = classifier
        
        # Extractors per tipo
        self.extractors = {
            'fattura': InvoiceExtractor(),
            'contratto': ContractExtractor(),
            'lettera': LetterExtractor(),
            'report': ReportExtractor(),
        }
    
    def process(self, documento: str) -> Dict:
        """
        Processa un documento end-to-end.
        
        Returns:
            {
                'tipo': str,
                'confidence_classificazione': float,
                'dati_estratti': dict,
                'validation': dict
            }
        """
        # Step 1: Classification
        X = self.vectorizer.transform([documento])
        
        # Predizione con probabilità
        tipo_predetto = self.classifier.predict(X)[0]
        probabilita = self.classifier.predict_proba(X)[0]
        confidence_class = max(probabilita)
        
        # Step 2: Extraction
        if tipo_predetto in self.extractors:
            extractor = self.extractors[tipo_predetto]
            dati = extractor.extract(documento)
            validation = extractor.validate(dati)
        else:
            dati = {}
            validation = {'valid': False, 'issues': ['Tipo non supportato']}
        
        # Step 3: Risultato
        return {
            'tipo': tipo_predetto,
            'confidence_classificazione': confidence_class,
            'dati_estratti': dati if hasattr(dati, '__dict__') else vars(dati) if hasattr(dati, '__dict__') else dati,
            'validation': validation,
            'richiede_review': confidence_class < 0.8 or not validation['valid']
        }

# Extractors semplificati per altri tipi
class ContractExtractor:
    def extract(self, testo):
        @dataclass
        class ContractData:
            parti: Optional[str] = None
            durata: Optional[str] = None
            valore: Optional[str] = None
            confidence: float = 0.0
        
        result = ContractData()
        
        # Pattern semplici
        parti = re.search(r'tra\s+(.+?)\s+e\s+(.+?)(?:\s+durata|\s+decorrenza|$)', testo, re.I)
        if parti:
            result.parti = f"{parti.group(1)} / {parti.group(2)}"
        
        durata = re.search(r'(\d+)\s*(?:mesi|anni|months|years)', testo, re.I)
        if durata:
            result.durata = durata.group(0)
        
        valore = re.search(r'€\s*[\d.,]+', testo)
        if valore:
            result.valore = valore.group(0)
        
        result.confidence = sum([1 for x in [result.parti, result.durata, result.valore] if x]) / 3
        return result
    
    def validate(self, data):
        return {'valid': data.confidence > 0.3, 'issues': []}

class LetterExtractor:
    def extract(self, testo):
        @dataclass
        class LetterData:
            destinatario: Optional[str] = None
            tipo_saluto: Optional[str] = None
            confidence: float = 0.5
        
        result = LetterData()
        
        saluto = re.search(r'(gentile|egregio|spett\.?le|dear)\s+([^,]+)', testo, re.I)
        if saluto:
            result.destinatario = saluto.group(2)
            result.tipo_saluto = saluto.group(1)
            result.confidence = 0.8
        
        return result
    
    def validate(self, data):
        return {'valid': True, 'issues': []}

class ReportExtractor:
    def extract(self, testo):
        @dataclass
        class ReportData:
            tipo_report: Optional[str] = None
            metriche: List = None
            confidence: float = 0.5
        
        result = ReportData()
        result.metriche = []
        
        # Tipo report
        tipo = re.search(r'(trimestrale|settimanale|annual|quarterly|weekly|mensile)', testo, re.I)
        if tipo:
            result.tipo_report = tipo.group(1)
        
        # Metriche numeriche
        metriche = re.findall(r'(\w+)\s*[:=]\s*([\d.,]+[%€]?|€[\d.,]+)', testo)
        result.metriche = metriche[:5]  # Max 5
        
        result.confidence = 0.6 if result.tipo_report else 0.4
        return result
    
    def validate(self, data):
        return {'valid': True, 'issues': []}

# Test pipeline
pipeline = DocumentIntelligencePipeline(vectorizer, classifier)

test_docs = [
    "FATTURA N. 2024/999 Data: 01/04/2024 Cliente: Test Srl Totale: €2.000,00",
    "CONTRATTO tra Alfa SpA e Beta Srl durata 24 mesi valore €100.000",
    "Gentile Dott. Rossi, Le scrivo per confermare l'appuntamento. Cordiali saluti",
    "Report Q1 2024 Fatturato: €1.5M Crescita: +20% Margine: 15%",
]

print("="*60)
print("DEMO 5: PIPELINE INTEGRATA")
print("="*60)

for doc in test_docs:
    print(f"\n{'='*60}")
    print(f"📄 Input: '{doc[:50]}...'")
    
    result = pipeline.process(doc)
    
    print(f"\n   📋 RISULTATO:")
    print(f"      Tipo: {result['tipo'].upper()}")
    print(f"      Confidence: {result['confidence_classificazione']:.1%}")
    print(f"      Review richiesta: {'Sì' if result['richiede_review'] else 'No'}")

---

# 4. Esercizi Svolti

## Esercizio 1: Sistema di Smistamento Email Automatico

**Obiettivo:** Creare un sistema che classifica email in arrivo e le smista al dipartimento corretto.

In [ ]:
# ============================================================
# ESERCIZIO 1 - SOLUZIONE
# Sistema di Smistamento Email Automatico
# ============================================================

class EmailRouter:
    """
    Sistema di routing automatico email.
    Classifica email e le assegna al dipartimento corretto.
    """
    
    # Mapping dipartimenti
    DEPARTMENTS = {
        'vendite': ['offerta', 'preventivo', 'acquisto', 'ordine', 'prezzo', 'listino'],
        'supporto': ['problema', 'errore', 'guasto', 'malfunzionamento', 'assistenza', 'aiuto'],
        'amministrazione': ['fattura', 'pagamento', 'bonifico', 'sollecito', 'scadenza'],
        'hr': ['candidatura', 'cv', 'curriculum', 'assunzione', 'colloquio', 'stipendio'],
        'legale': ['contratto', 'clausola', 'avvocato', 'causa', 'reclamo', 'diffida'],
    }
    
    def __init__(self):
        self.vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=500)
        self.classifier = LogisticRegression(max_iter=1000)
        self.is_trained = False
    
    def train(self, emails: List[Dict]):
        """Addestra il classificatore su email etichettate."""
        X = [e['testo'] for e in emails]
        y = [e['dipartimento'] for e in emails]
        
        X_tfidf = self.vectorizer.fit_transform(X)
        self.classifier.fit(X_tfidf, y)
        self.is_trained = True
    
    def route(self, email: str) -> Dict:
        """
        Classifica e instrada un'email.
        
        Returns:
            {
                'dipartimento': str,
                'confidence': float,
                'keywords_rilevate': list,
                'priorita': str
            }
        """
        if not self.is_trained:
            # Fallback: routing basato su keyword
            return self._keyword_routing(email)
        
        # ML-based routing
        X = self.vectorizer.transform([email])
        dept = self.classifier.predict(X)[0]
        proba = max(self.classifier.predict_proba(X)[0])
        
        # Analisi priorità
        priorita = self._detect_priority(email)
        
        # Keywords trovate
        keywords = self._find_keywords(email, dept)
        
        return {
            'dipartimento': dept,
            'confidence': proba,
            'keywords_rilevate': keywords,
            'priorita': priorita
        }
    
    def _keyword_routing(self, email: str) -> Dict:
        """Routing basato su keyword (fallback)."""
        email_lower = email.lower()
        scores = {}
        
        for dept, keywords in self.DEPARTMENTS.items():
            score = sum(1 for kw in keywords if kw in email_lower)
            if score > 0:
                scores[dept] = score
        
        if scores:
            best_dept = max(scores, key=scores.get)
            return {
                'dipartimento': best_dept,
                'confidence': min(scores[best_dept] / 3, 1.0),
                'keywords_rilevate': [kw for kw in self.DEPARTMENTS[best_dept] if kw in email_lower],
                'priorita': self._detect_priority(email)
            }
        
        return {
            'dipartimento': 'generale',
            'confidence': 0.0,
            'keywords_rilevate': [],
            'priorita': 'normale'
        }
    
    def _detect_priority(self, email: str) -> str:
        """Rileva priorità dell'email."""
        email_lower = email.lower()
        
        if any(w in email_lower for w in ['urgente', 'urgent', 'immediato', 'asap']):
            return 'ALTA'
        if any(w in email_lower for w in ['sollecito', 'scadenza', 'deadline']):
            return 'MEDIA'
        return 'NORMALE'
    
    def _find_keywords(self, email: str, dept: str) -> List[str]:
        """Trova keywords per il dipartimento."""
        if dept not in self.DEPARTMENTS:
            return []
        return [kw for kw in self.DEPARTMENTS[dept] if kw in email.lower()]

# Training data
training_emails = [
    {'testo': 'Richiedo un preventivo per 100 licenze software', 'dipartimento': 'vendite'},
    {'testo': 'Vorrei un\'offerta per servizi cloud', 'dipartimento': 'vendite'},
    {'testo': 'Il sistema dà errore quando tento il login', 'dipartimento': 'supporto'},
    {'testo': 'Ho un problema con l\'applicazione mobile', 'dipartimento': 'supporto'},
    {'testo': 'Non ho ricevuto la fattura del mese scorso', 'dipartimento': 'amministrazione'},
    {'testo': 'Sollecito pagamento fattura scaduta', 'dipartimento': 'amministrazione'},
    {'testo': 'Invio la mia candidatura per la posizione aperta', 'dipartimento': 'hr'},
    {'testo': 'Allego il mio CV per la vostra valutazione', 'dipartimento': 'hr'},
    {'testo': 'Richiedo revisione del contratto di fornitura', 'dipartimento': 'legale'},
    {'testo': 'Diffida per mancato rispetto clausole contrattuali', 'dipartimento': 'legale'},
]

# Train e test
router = EmailRouter()
router.train(training_emails)

test_emails = [
    "URGENTE: Problema critico con il server di produzione, non funziona nulla!",
    "Gentili, vorrei richiedere un preventivo per il vostro prodotto enterprise",
    "Allego il bonifico per saldo fattura 2024/123",
    "Buongiorno, sono interessato alla posizione di Data Analyst pubblicata",
]

print("="*60)
print("ESERCIZIO 1: SISTEMA SMISTAMENTO EMAIL")
print("="*60)

for email in test_emails:
    print(f"\n📧 Email: '{email[:50]}...'")
    
    result = router.route(email)
    
    print(f"   📁 Dipartimento: {result['dipartimento'].upper()}")
    print(f"   📊 Confidence: {result['confidence']:.1%}")
    print(f"   🏷️ Keywords: {result['keywords_rilevate']}")
    print(f"   ⚡ Priorità: {result['priorita']}")

---

## Esercizio 2: CV Parser per HR

**Obiettivo:** Estrarre informazioni strutturate da curriculum vitae (nome, competenze, esperienze, formazione).

In [ ]:
# ============================================================
# ESERCIZIO 2 - SOLUZIONE
# CV Parser per HR
# ============================================================

class CVParser:
    """
    Parser per estrarre informazioni strutturate da CV.
    
    Estrae:
    - Dati personali (nome, email, telefono)
    - Competenze tecniche
    - Esperienze lavorative
    - Formazione
    """
    
    def __init__(self):
        # Competenze comuni da cercare
        self.tech_skills = [
            'python', 'java', 'javascript', 'sql', 'excel', 'power bi',
            'machine learning', 'data analysis', 'spark', 'hadoop',
            'aws', 'azure', 'docker', 'kubernetes', 'git',
            'r', 'tableau', 'sap', 'salesforce', 'mongodb'
        ]
        
        self.soft_skills = [
            'leadership', 'teamwork', 'comunicazione', 'problem solving',
            'project management', 'agile', 'scrum'
        ]
        
        # Pattern
        self.patterns = {
            'email': r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
            'telefono': r'(?:\+39\s?)?(?:0\d{1,4}|\d{3})[\s.-]?\d{6,7}',
            'laurea': r'laurea\s+(?:triennale|magistrale|in)?\s*(?:in\s+)?([a-zA-ZÀ-ÿ\s]+)',
            'esperienza_anni': r'(\d+)\s*(?:\+)?\s*anni?\s*(?:di)?\s*esperienza',
        }
    
    def parse(self, cv_text: str) -> Dict:
        """Estrae tutte le informazioni dal CV."""
        cv_lower = cv_text.lower()
        
        result = {
            'contatti': self._extract_contacts(cv_text),
            'nome': self._extract_name(cv_text),
            'competenze_tecniche': self._extract_skills(cv_lower, self.tech_skills),
            'soft_skills': self._extract_skills(cv_lower, self.soft_skills),
            'formazione': self._extract_education(cv_text),
            'anni_esperienza': self._extract_experience_years(cv_lower),
            'score_match': 0.0  # Calcolato dopo
        }
        
        return result
    
    def _extract_contacts(self, text: str) -> Dict:
        """Estrae informazioni di contatto."""
        contacts = {'email': None, 'telefono': None}
        
        email = re.search(self.patterns['email'], text)
        if email:
            contacts['email'] = email.group(0)
        
        phone = re.search(self.patterns['telefono'], text)
        if phone:
            contacts['telefono'] = phone.group(0)
        
        return contacts
    
    def _extract_name(self, text: str) -> Optional[str]:
        """Estrae il nome (prima riga o pattern)."""
        lines = text.strip().split('\n')
        for line in lines[:3]:  # Prime 3 righe
            line = line.strip()
            # Pattern: 2-3 parole titlecase
            if re.match(r'^[A-Z][a-zÀ-ÿ]+(?:\s+[A-Z][a-zÀ-ÿ]+){1,2}$', line):
                return line
        return None
    
    def _extract_skills(self, text: str, skill_list: List[str]) -> List[str]:
        """Trova skill presenti nel testo."""
        found = []
        for skill in skill_list:
            if skill in text:
                found.append(skill)
        return found
    
    def _extract_education(self, text: str) -> List[str]:
        """Estrae informazioni sulla formazione."""
        education = []
        
        # Cerca lauree
        lauree = re.findall(self.patterns['laurea'], text, re.IGNORECASE)
        for l in lauree:
            education.append(f"Laurea in {l.strip()}")
        
        # Cerca certificazioni
        cert_keywords = ['certificat', 'certified', 'certification']
        for line in text.split('\n'):
            if any(kw in line.lower() for kw in cert_keywords):
                education.append(line.strip()[:50])
        
        return education[:5]  # Max 5
    
    def _extract_experience_years(self, text: str) -> Optional[int]:
        """Estrae anni di esperienza."""
        match = re.search(self.patterns['esperienza_anni'], text)
        if match:
            return int(match.group(1))
        return None
    
    def match_job(self, cv_data: Dict, job_requirements: Dict) -> float:
        """
        Calcola score di matching tra CV e requisiti job.
        
        Args:
            cv_data: dati estratti dal CV
            job_requirements: {
                'skills_required': list,
                'min_experience': int,
                'education': str
            }
        """
        score = 0.0
        max_score = 0.0
        
        # Match competenze (peso 50%)
        required_skills = set(s.lower() for s in job_requirements.get('skills_required', []))
        cv_skills = set(cv_data.get('competenze_tecniche', []))
        
        if required_skills:
            skill_match = len(required_skills & cv_skills) / len(required_skills)
            score += skill_match * 50
            max_score += 50
        
        # Match esperienza (peso 30%)
        min_exp = job_requirements.get('min_experience', 0)
        cv_exp = cv_data.get('anni_esperienza', 0) or 0
        
        if min_exp > 0:
            exp_match = min(cv_exp / min_exp, 1.0)
            score += exp_match * 30
            max_score += 30
        
        # Formazione (peso 20%)
        if cv_data.get('formazione'):
            score += 20
        max_score += 20
        
        return score / max_score if max_score > 0 else 0.0

# Test
parser = CVParser()

cv_esempio = """
Mario Rossi
Data Scientist | Milano

Email: mario.rossi@email.com
Telefono: +39 333 1234567

COMPETENZE TECNICHE
Python, SQL, Machine Learning, Power BI, Excel
AWS, Docker, Spark

ESPERIENZA
5+ anni di esperienza in analisi dati e machine learning

FORMAZIONE
Laurea Magistrale in Informatica - Politecnico di Milano
Certificazione AWS Solutions Architect
"""

job_requirements = {
    'skills_required': ['python', 'sql', 'machine learning', 'aws'],
    'min_experience': 3,
    'education': 'laurea'
}

print("="*60)
print("ESERCIZIO 2: CV PARSER")
print("="*60)

# Parse CV
cv_data = parser.parse(cv_esempio)

print("\n📄 CV ANALIZZATO:")
print("-" * 40)
print(f"Nome: {cv_data['nome']}")
print(f"Email: {cv_data['contatti']['email']}")
print(f"Telefono: {cv_data['contatti']['telefono']}")
print(f"Competenze tecniche: {', '.join(cv_data['competenze_tecniche'])}")
print(f"Anni esperienza: {cv_data['anni_esperienza']}")
print(f"Formazione: {cv_data['formazione']}")

# Match con job
match_score = parser.match_job(cv_data, job_requirements)
cv_data['score_match'] = match_score

print(f"\n{'='*60}")
print("JOB MATCHING")
print("="*60)
print(f"Requisiti posizione:")
print(f"  Skills: {job_requirements['skills_required']}")
print(f"  Esperienza minima: {job_requirements['min_experience']} anni")
print(f"\n🎯 MATCH SCORE: {match_score:.0%}")

# Valutazione
if match_score >= 0.8:
    print("   ✅ ALTAMENTE COMPATIBILE - Convocare a colloquio")
elif match_score >= 0.5:
    print("   ⚠️ PARZIALMENTE COMPATIBILE - Valutare")
else:
    print("   ❌ NON COMPATIBILE - Scartare")

---
## Esercizio 3: Sistema di Archiviazione Automatica

**Obiettivo:** Costruire un sistema completo che classifica documenti in arrivo e li archivia in cartelle appropriate, estraendo metadati per un database.

In [ ]:
# ============================================================
# ESERCIZIO 3 - SOLUZIONE
# Sistema di Archiviazione Automatica Documenti
# ============================================================

import os
from datetime import datetime

class DocumentArchiver:
    """
    Sistema di archiviazione automatica documenti.
    
    Workflow:
    1. Riceve documento in ingresso
    2. Classifica tipologia
    3. Estrae metadati rilevanti
    4. Determina cartella di destinazione
    5. Registra in database (simulato)
    """
    
    def __init__(self, base_path: str = "./archivio"):
        self.base_path = base_path
        self.pipeline = DocumentIntelligencePipeline()  # Da demo precedente
        
        # Mapping categoria -> cartella
        self.folder_mapping = {
            'fattura': 'Fatture',
            'contratto': 'Contratti',
            'lettera': 'Corrispondenza',
            'report': 'Report'
        }
        
        # Database simulato
        self.document_database = []
    
    def process_document(self, doc_text: str, filename: str) -> Dict:
        """
        Processa un documento: classifica, estrae metadati, archivia.
        
        Returns:
            Dict con risultato elaborazione
        """
        # Step 1: Classifica
        classification = self.pipeline.classify(doc_text)
        doc_type = classification['tipologia']
        
        # Step 2: Estrai metadati
        extraction = self.pipeline.extract(doc_text)
        
        # Step 3: Determina destinazione
        folder = self.folder_mapping.get(doc_type, 'Altri')
        
        # Step 4: Genera record per database
        record = {
            'id': len(self.document_database) + 1,
            'filename_originale': filename,
            'tipologia': doc_type,
            'confidence': classification['confidence'],
            'cartella': folder,
            'metadati': extraction,
            'data_elaborazione': datetime.now().isoformat(),
            'status': 'archiviato'
        }
        
        # Step 5: Salva in database
        self.document_database.append(record)
        
        return record
    
    def get_archive_stats(self) -> Dict:
        """Statistiche sull'archivio."""
        if not self.document_database:
            return {'totale': 0}
        
        stats = {
            'totale': len(self.document_database),
            'per_tipologia': {},
            'confidence_media': 0.0
        }
        
        confidences = []
        for doc in self.document_database:
            tipo = doc['tipologia']
            stats['per_tipologia'][tipo] = stats['per_tipologia'].get(tipo, 0) + 1
            confidences.append(doc['confidence'])
        
        stats['confidence_media'] = np.mean(confidences)
        
        return stats
    
    def search_documents(self, query: str = None, tipologia: str = None) -> List[Dict]:
        """Cerca documenti nell'archivio."""
        results = self.document_database.copy()
        
        if tipologia:
            results = [d for d in results if d['tipologia'] == tipologia]
        
        # Ricerca semplice nei metadati
        if query:
            query_lower = query.lower()
            filtered = []
            for doc in results:
                metadati_str = str(doc.get('metadati', {})).lower()
                if query_lower in metadati_str:
                    filtered.append(doc)
            results = filtered
        
        return results

# Test del sistema
archiver = DocumentArchiver()

# Documenti di test
documenti_test = [
    {
        'filename': 'fattura_2024_001.txt',
        'content': """
        FATTURA N. 2024/001
        Data: 15/01/2024
        Fornitore: TechSupply Srl
        Totale: EUR 1.250,00
        IVA inclusa
        """
    },
    {
        'filename': 'contratto_servizi.txt',
        'content': """
        CONTRATTO DI SERVIZI
        Le parti concordano quanto segue:
        Durata: 24 mesi
        Importo mensile: EUR 500
        Decorrenza: 01/02/2024
        """
    },
    {
        'filename': 'report_q1.txt',
        'content': """
        REPORT TRIMESTRALE Q1 2024
        Analisi performance vendite
        Crescita: +15% vs Q1 2023
        Obiettivi raggiunti: 95%
        """
    },
    {
        'filename': 'lettera_cliente.txt',
        'content': """
        Gentile Cliente,
        La ringraziamo per la fiducia.
        Cordiali saluti,
        Servizio Clienti
        """
    },
    {
        'filename': 'fattura_2024_002.txt',
        'content': """
        FATTURA N. 2024/002
        Data: 20/01/2024
        Cliente: ABC Corp
        Importo: EUR 3.500,00
        """
    }
]

print("="*60)
print("ESERCIZIO 3: SISTEMA ARCHIVIAZIONE AUTOMATICA")
print("="*60)

print("\n📁 ELABORAZIONE DOCUMENTI IN ARRIVO")
print("-" * 50)

for doc in documenti_test:
    result = archiver.process_document(doc['content'], doc['filename'])
    print(f"\n✅ {doc['filename']}")
    print(f"   Tipo: {result['tipologia']} (conf: {result['confidence']:.0%})")
    print(f"   Cartella: {result['cartella']}/")
    print(f"   ID archivio: #{result['id']}")

# Statistiche
print(f"\n{'='*60}")
print("STATISTICHE ARCHIVIO")
print("="*60)

stats = archiver.get_archive_stats()
print(f"\n📊 Documenti totali archiviati: {stats['totale']}")
print(f"   Confidence media: {stats['confidence_media']:.1%}")
print("\nDistribuzione per tipologia:")
for tipo, count in stats['per_tipologia'].items():
    print(f"   - {tipo.capitalize()}: {count}")

# Ricerca
print(f"\n{'='*60}")
print("TEST RICERCA")
print("="*60)

# Cerca fatture
fatture = archiver.search_documents(tipologia='fattura')
print(f"\n🔍 Ricerca: tipologia='fattura'")
print(f"   Trovati: {len(fatture)} documenti")

# Cerca per importo
euro_docs = archiver.search_documents(query='eur')
print(f"\n🔍 Ricerca: query='eur'")
print(f"   Trovati: {len(euro_docs)} documenti")

---
---
# 6. Conclusione Operativa

## Cosa abbiamo imparato

**Document Intelligence** è l'applicazione pratica di NLP per automatizzare il trattamento documentale:

| Componente | Tecnica | Applicazione |
|------------|---------|--------------|
| Classificazione | TF-IDF + ML | Smistamento automatico |
| Estrazione | Regex + Pattern | Dati strutturati da testo |
| Validazione | Regole business | Quality assurance |
| Pipeline | Integrazione | Workflow end-to-end |

## Pattern Architetturali

1. **Pipeline Modulare**: Classificazione → Estrazione → Validazione → Output
2. **Strategy Pattern**: Estrattori specifici per ogni tipo documento
3. **Dataclass Output**: Risultati strutturati e tipizzati

## Metriche di Successo

Per valutare un sistema di Document Intelligence:

- **Accuracy classificazione**: > 90% per produzione
- **Precision estrazione**: Dati estratti corretti
- **Recall estrazione**: Dati trovati vs esistenti
- **Tempo elaborazione**: Secondi per documento
- **Copertura**: % campi estratti con successo

## Quando Usare Document Intelligence

✅ **Ideale per:**
- Documenti semi-strutturati (fatture, contratti, form)
- Volumi elevati ripetitivi
- Estrazione campi prevedibili

⚠️ **Richiede cautela per:**
- Documenti molto variabili
- Layout complessi (tabelle nested)
- Richieste di comprensione semantica profonda

---
# 📋 BIGNAMI - Document Intelligence

## Architettura Pipeline Base

```
DOCUMENTO → Preprocessing → Classificazione → Estrazione → Validazione → OUTPUT
                              |                    |
                              v                    v
                         TF-IDF + ML         Regex/Pattern
```

## Template Estrattore

```python
from dataclasses import dataclass
from typing import Optional, Dict, List
import re

@dataclass
class ExtractedData:
    campo_1: Optional[str] = None
    campo_2: Optional[float] = None
    raw_text: str = ""

class DocumentExtractor:
    def __init__(self):
        self.patterns = {
            'campo_1': r'pattern_regex_qui',
            'campo_2': r'\d+[.,]\d{2}'
        }
    
    def extract(self, text: str) -> ExtractedData:
        result = ExtractedData(raw_text=text)
        for field, pattern in self.patterns.items():
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                setattr(result, field, match.group(1))
        return result
```

## Template Classificatore

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

classifier = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('clf', MultinomialNB())
])
classifier.fit(X_train, y_train)
prediction = classifier.predict([nuovo_documento])[0]
```

## Pattern Regex Comuni

| Tipo | Pattern |
|------|---------|
| Email | `r'\b[\w.+-]+@[\w-]+\.[\w.-]+\b'` |
| Telefono IT | `r'(?:\+39)?\s*\d{2,4}[\s.-]?\d{6,7}'` |
| Data IT | `r'\d{1,2}[/.-]\d{1,2}[/.-]\d{2,4}'` |
| Importo EUR | `r'(?:EUR\|€)?\s*[\d.,]+(?:\s*(?:EUR\|€))?'` |
| Partita IVA | `r'\b\d{11}\b'` |
| Codice Fiscale | `r'\b[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]\b'` |

## Checklist Progetto IDP

- [ ] Definire tipologie documento
- [ ] Raccogliere dataset rappresentativo
- [ ] Identificare campi da estrarre per tipo
- [ ] Creare pattern regex per ogni campo
- [ ] Trainare classificatore
- [ ] Implementare estrattori specifici
- [ ] Aggiungere validazione output
- [ ] Testare su documenti reali
- [ ] Calcolare metriche accuracy/precision/recall

## Errori Comuni

| Errore | Soluzione |
|--------|-----------|
| Regex troppo rigida | Usare `?` e `*` per parti opzionali |
| Encoding errato | Specificare `encoding='utf-8'` |
| Falsi positivi | Aggiungere contesto nel pattern |
| Campi mancanti | Return None, non errore |
| Parsing numeri | Gestire separatori `,` e `.` |